Este script lee los datos del archivo train store.csv y en base a la venta, entrena 5 modelos diferentes de ML con diferentes hiperparametros
Escoge el mejor modelo menor MSE y MAE y prueba los datos del archivo train.csv


In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit, ParameterGrid
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
import time

# Definir los modelos con los hiperparámetros que deseas ajustar
modelos = {
    'Decision Tree': DecisionTreeRegressor(),
    'KNN': KNeighborsRegressor(),
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(),
    'XGBoost': XGBRegressor()
}

# Definir los hiperparámetros que deseas ajustar para cada modelo
hiperparametros = {
    'Decision Tree': {'max_depth': [3, 5, 7]},
    'KNN': {'n_neighbors': [3, 5, 7]},
    'Linear Regression': {},
    'Random Forest': {'n_estimators': [50, 100, 150]},
    'XGBoost': {'n_estimators': [50, 100, 150], 'max_depth': [3, 5, 7]}
}

# Leer datos y convertir a tipo fecha
df = pd.read_csv("train store.csv")
df['date'] = pd.to_datetime(df['date'])

# ordenar por fecha
df = df.sort_values(by=['date', 'store', 'item'])

# Agregar columnas de día, mes y año
df['day'] = df['date'].dt.day
df['day_of_week'] = df['date'].dt.dayofweek
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

# Calcular 7 lags temporales y eliminar nulos 
max_lags = 7
for i in range(1, max_lags + 1):
    df[f'sales_daylag{i}'] = df.groupby(['store', 'item'])['sales'].shift(i)
df.dropna(inplace=True)
df = df.drop(['date'], axis=1)

# Convertir datos categoricos a numericos
categorical_cols = ['store', 'item']
encoder = OneHotEncoder(sparse_output=False, drop='first')
encoded_cols = encoder.fit_transform(df[categorical_cols])
encoded_df = pd.DataFrame(encoded_cols, columns=encoder.get_feature_names_out(categorical_cols))
df = df.drop(columns=categorical_cols)
df = pd.concat([df, encoded_df], axis=1)
df = df.dropna()

# Crear splits temporales para evaluacion de los modelos
n_splits = 5
tscv = TimeSeriesSplit(n_splits=n_splits)

# Inicializar lista de resultados
resultados = []

# Iterar sobre cada modelo
for nombre_modelo, modelo in modelos.items():
    print('Entrenando modelo:', nombre_modelo)
    # Inicializar la mejor puntuación para este modelo con un valor grande
    mejor_modelo = ''
    mejor_modelo_nombre = ''
    mejor_puntuacion = float('inf') 

    # Iterar sobre cada combinación de hiperparámetros
    for params in ParameterGrid(hiperparametros[nombre_modelo]):
        print('Con los siguientes Hiperparametros:', params)
        # Configurar el modelo con los hiperparámetros actuales
        modelo.set_params(**params)

        # Inicializar listas para guardar las puntuaciones de MSE y MAE para cada split
        mse_scores = []
        mae_scores = []
        start_time = time.time()

        # Iterar sobre cada split temporal
        for train_index, test_index in tscv.split(df):
            # Dividir los datos en conjunto de entrenamiento y prueba
            train = df.iloc[train_index]
            test = df.iloc[test_index]

            X_train = train.drop('sales', axis=1)
            y_train = train['sales']
            X_test = test.drop('sales', axis=1)
            y_test = test['sales']

            # Entrenar y evaluar el modelo
            modelo.fit(X_train, y_train)
            y_pred = modelo.predict(X_test)

            mse = np.sqrt(mean_squared_error(y_test, y_pred))
            mae = mean_absolute_error(y_test, y_pred)

            mse_scores.append(mse)
            mae_scores.append(mae)

        # Calcular la puntuación media de MSE y MAE para este modelo y estos hiperparámetros y el tiempo de ejecucion
        train_time = time.time() - start_time
        mse_mean = np.mean(mse_scores)
        mae_mean = np.mean(mae_scores)

        # Si encontramos una puntuación mejor, actualizar el mejor set de hiperparametros y la mejor puntuación
        if mse_mean < mejor_puntuacion:
            mejor_modelo = modelo
            mejor_modelo_nombre = nombre_modelo
            mejor_puntuacion = mse_mean
            mejor_hiperparametros = params
            mejor_tiempo = train_time

    # Guardar en la lista de resultados el mejor set de hiperparametros por modelo
    resultados.append({
        'Modelo': mejor_modelo,
        'Modelo Nombre': mejor_modelo_nombre, 
        'RMSE': mejor_puntuacion, 
        'MAE': mae_mean, 
        'Mejor Hiperparámetro': mejor_hiperparametros, 
        'Tiempo de Entrenamiento:': mejor_tiempo
        })

# Mostrar los resultados
df_resultados = pd.DataFrame(resultados)

# Ordenar los resultados por MSE y MAE en orden ascendente
df_resultados = df_resultados.sort_values(by=['RMSE', 'MAE'])
print(df_resultados)

# Obtener el mejor modelo y sus hiperparámetros
mejor_modelo_info = df_resultados.iloc[0]

# Extraer el nombre del mejor modelo y sus hiperparámetros
mejor_modelo_final = mejor_modelo_info['Modelo']
mejor_modelo_nombre_final = mejor_modelo_info['Modelo Nombre']
mejor_hiperparametros_final = mejor_modelo_info['Mejor Hiperparámetro']

# Leer datos del archivo test y dar mismo trato que al train
df_test = pd.read_csv("train store.csv")  # AQUI FLTA PONER EL ARCHIVO DE TEST
df_test['date'] = pd.to_datetime(df_test['date'])

# ordenar por fecha
df_test = df_test.sort_values(by=['date', 'store', 'item'])

# Agregar columnas de día, mes y año
df_test['day'] = df_test['date'].dt.day
df_test['day_of_week'] = df_test['date'].dt.dayofweek
df_test['month'] = df_test['date'].dt.month
df_test['year'] = df_test['date'].dt.year

# Calcular 7 lags temporales y eliminar nulos 
max_lags = 7
for j in range(1, max_lags + 1):
    df_test[f'sales_daylag{j}'] = df_test.groupby(['store', 'item'])['sales'].shift(j)
df_test.dropna(inplace=True)
df_test = df_test.drop(['date'], axis=1)

# Convertir datos categoricos a numericos
categorical_cols = ['store', 'item']
encoder = OneHotEncoder(sparse_output=False, drop='first')
encoded_cols = encoder.fit_transform(df_test[categorical_cols])
encoded_df_test = pd.DataFrame(encoded_cols, columns=encoder.get_feature_names_out(categorical_cols))
df_test = df_test.drop(columns=categorical_cols)
df_test = pd.concat([df_test, encoded_df_test], axis=1)
df_test = df_test.dropna()

X_train_final = df.drop('sales', axis=1)
y_train_final = df['sales']
X_test_final = df_test.drop('sales', axis=1)
y_test_final = df_test['sales']

# Entrenar y evaluar el modelo
modelo_final, = mejor_modelo_final  # Desempaqueta la tupla para obtener el modelo
modelo_final.set_params(**mejor_hiperparametros_final)
modelo_final.fit(X_train_final, y_train_final)
y_pred_final = modelo_final.predict(X_test_final)

mse_final = np.sqrt(mean_squared_error(y_test_final, y_pred_final))
mae_final = mean_absolute_error(y_test_final, y_pred_final)

print(f'El mejor modelo es {mejor_modelo_nombre_final}, tiene los hiperparametros {mejor_hiperparametros_final}, y con el archivo test tiene un rmse {mse_final} y un mae de {mae_final}')

Entrenando modelo: Decision Tree
Con los siguientes Hiperparametros: {'max_depth': 3}
Con los siguientes Hiperparametros: {'max_depth': 5}
Con los siguientes Hiperparametros: {'max_depth': 7}
Entrenando modelo: KNN
Con los siguientes Hiperparametros: {'n_neighbors': 3}
Con los siguientes Hiperparametros: {'n_neighbors': 5}
Con los siguientes Hiperparametros: {'n_neighbors': 7}
Entrenando modelo: Linear Regression
Con los siguientes Hiperparametros: {}
Entrenando modelo: Random Forest
Con los siguientes Hiperparametros: {'n_estimators': 50}
Con los siguientes Hiperparametros: {'n_estimators': 100}
Con los siguientes Hiperparametros: {'n_estimators': 150}
Entrenando modelo: XGBoost
Con los siguientes Hiperparametros: {'max_depth': 3, 'n_estimators': 50}
Con los siguientes Hiperparametros: {'max_depth': 3, 'n_estimators': 100}
Con los siguientes Hiperparametros: {'max_depth': 3, 'n_estimators': 150}
Con los siguientes Hiperparametros: {'max_depth': 5, 'n_estimators': 50}
Con los siguiente

In [21]:
# Leer datos y convertir a tipo fecha
df = pd.read_csv("train store.csv")
df['date'] = pd.to_datetime(df['date'])

# ordenar por fecha
df = df.sort_values(by=['date', 'store', 'item'])

# Agregar columnas de día, mes y año
df['day'] = df['date'].dt.day
df['day_of_week'] = df['date'].dt.dayofweek
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

# Calcular 7 lags temporales y eliminar nulos 
max_lags = 7
for i in range(1, max_lags + 1):
    df[f'sales_daylag{i}'] = df.groupby(['store', 'item'])['sales'].shift(i)
df.dropna(inplace=True)
df = df.drop(['date'], axis=1)

# Convertir datos categoricos a numericos
categorical_cols = ['store', 'item']
encoder = OneHotEncoder(sparse_output=False, drop='first')
encoded_cols = encoder.fit_transform(df[categorical_cols])
encoded_df = pd.DataFrame(encoded_cols, columns=encoder.get_feature_names_out(categorical_cols))
df = df.drop(columns=categorical_cols)
df = pd.concat([df, encoded_df], axis=1)
df = df.dropna()

# Leer datos del archivo test y dar mismo trato que al train
df_test = pd.read_csv("test.csv")  
df_test['date'] = pd.to_datetime(df_test['date'])

# ordenar por fecha
df_test = df_test.sort_values(by=['date', 'store', 'item'])

# Agregar columnas de día, mes y año
df_test['day'] = df_test['date'].dt.day
df_test['day_of_week'] = df_test['date'].dt.dayofweek
df_test['month'] = df_test['date'].dt.month
df_test['year'] = df_test['date'].dt.year

# Convertir datos categoricos a numericos
categorical_cols = ['store', 'item']
encoder = OneHotEncoder(sparse_output=False, drop='first')
encoded_cols = encoder.fit_transform(df_test[categorical_cols])
encoded_df_test = pd.DataFrame(encoded_cols, columns=encoder.get_feature_names_out(categorical_cols))
df_test = df_test.drop(columns=categorical_cols)
df_test = pd.concat([df_test, encoded_df_test], axis=1)
df_test = df_test.dropna()

# Entrenar modelo final
X_train_final = df.drop('sales', axis=1)
y_train_final = df['sales']
modelo_final, = mejor_modelo_final  
modelo_final.set_params(**mejor_hiperparametros_final)
modelo_final.fit(X_train_final, y_train_final)
y_pred_final = modelo_final.predict(df_test)

# Desplegar resultados y mandar a un archivo CSV
resultado = pd.DataFrame(y_pred_final)
print(resultado)

resultado.to_csv('resultado.csv')

ValueError: Feature shape mismatch, expected: 69, got 64